In [13]:
import os
import pickle
import base64
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import pandas as pd




In [14]:
# Functions
def get_doc_content(service, document_id):
    document = service.documents().get(documentId=document_id).execute()
    doc_content = document.get('body').get('content')
    return doc_content

def extract_subject_and_body(doc_content):
    # Add your implementation to extract email_subject and email_body from doc_content
    # ...
    # Return email_subject and email_body as strings
    return email_subject, email_body

def create_message(to, subject, message_text):
    message = f"To: {to}\nSubject: {subject}\n\n{message_text}"
    return {'raw': base64.urlsafe_b64encode(message.encode()).decode()}

# Establish GoogleAPI Scopes and Instance
SCOPES = ['https://www.googleapis.com/auth/gmail.compose', 'https://www.googleapis.com/auth/documents.readonly']
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('client_secret_600207218247-88idqn5h1tadq1d2gl7tkn826l63ud41.apps.googleusercontent.com.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

In [15]:
# Extract email content from Google Doc
doc_service = build('docs', 'v1', credentials=creds)
# The document id is found between the /d and the /edit portions of the GoogleDoc URL 
# Here is an example URL https://docs.google.com/document/d/13_xSd2YaiEP_p1DJl0A5gnsg2z0ahaN27BAtXfWcw2M/edit
document_id = '13_xSd2YaiEP_p1DJl0A5gnsg2z0ahaN27BAtXfWcw2M'
doc_content = get_doc_content(doc_service, document_id)
email_subject, email_body = extract_subject_and_body(doc_content)

# Read names and emails from CSV file
data = pd.read_csv('unmatched_names.csv')

# Sending personalized emails
for index, row in unmatched_names.iterrows():
    recipient_email = row["Email"]  # Extract email from the "Email" column
    personalized_email_body = email_body.replace("{name}", row["First"] + " " + row["Last"])
    message = f'To: {recipient_email}\nSubject: {email_subject}\n\n{personalized_email_body}'
    message_bytes = message.encode('utf-8')
    encoded_message = base64.urlsafe_b64encode(message_bytes).decode('utf-8')

    email = {'raw': encoded_message}

    # Try to send the email, print error content if there is an HttpError
    try:
        gmail_service.users().messages().send(userId='me', body=email).execute()
    except HttpError as error:
        print(error.content)

NameError: name 'unmatched_names' is not defined